<a href="https://colab.research.google.com/github/Sayeli/Optimized-CNN-Model/blob/master/Optimize_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


CNN Model with Keras Tuner

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 450kB 36.4MB/s 
     |████████████████████████████████| 3.8MB 36.9MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=f3073532e79667c8f4294192d43e3c9685fc4aec06d0edf5cd1f28dd88849046
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
ERROR: tensorflow 2.1.0rc2 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.3.3 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [5]:
print(tf.__version__)

2.1.0-rc2


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
train_images=train_images/255.0


In [0]:
test_images=test_images/255.0

In [12]:
train_images[0]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000

In [13]:
train_images[0].shape

(28, 28)

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=8,
                            directory='output',
                            project_name="Mnist Fashion")

In [20]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 16s 293us/sample - loss: 2.3045 - accuracy: 0.0980 - val_loss: 2.3039 - val_accuracy: 0.0925
Epoch 2/3
54000/54000 [==============================] - 10s 183us/sample - loss: 2.3038 - accuracy: 0.1021 - val_loss: 2.3038 - val_accuracy: 0.0973
Epoch 3/3
54000/54000 [==============================] - 9s 171us/sample - loss: 2.3039 - accuracy: 0.0992 - val_loss: 2.3038 - val_accuracy: 0.0985


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 186us/sample - loss: 2.3028 - accuracy: 0.0997 - val_loss: 2.3028 - val_accuracy: 0.0942
Epoch 2/3
54000/54000 [==============================] - 10s 176us/sample - loss: 2.3028 - accuracy: 0.0984 - val_loss: 2.3028 - val_accuracy: 0.0942
Epoch 3/3
54000/54000 [==============================] - 9s 171us/sample - loss: 2.3027 - accuracy: 0.1005 - val_loss: 2.3028 - val_accuracy: 0.0925


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 178us/sample - loss: 2.3042 - accuracy: 0.1005 - val_loss: 2.3039 - val_accuracy: 0.0973
Epoch 2/3
54000/54000 [==============================] - 9s 168us/sample - loss: 2.3041 - accuracy: 0.0999 - val_loss: 2.3025 - val_accuracy: 0.1050
Epoch 3/3
54000/54000 [==============================] - 9s 163us/sample - loss: 2.3038 - accuracy: 0.1004 - val_loss: 2.3044 - val_accuracy: 0.0925


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 180us/sample - loss: 0.7710 - accuracy: 0.7131 - val_loss: 0.5578 - val_accuracy: 0.7818
Epoch 2/3
54000/54000 [==============================] - 9s 172us/sample - loss: 0.5298 - accuracy: 0.8037 - val_loss: 0.5211 - val_accuracy: 0.8077
Epoch 3/3
54000/54000 [==============================] - 9s 171us/sample - loss: 0.4636 - accuracy: 0.8349 - val_loss: 0.4375 - val_accuracy: 0.8390


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 182us/sample - loss: 2.3041 - accuracy: 0.1019 - val_loss: 2.3029 - val_accuracy: 0.1027
Epoch 2/3
54000/54000 [==============================] - 10s 177us/sample - loss: 2.3039 - accuracy: 0.1002 - val_loss: 2.3040 - val_accuracy: 0.1027
Epoch 3/3
54000/54000 [==============================] - 9s 173us/sample - loss: 2.3040 - accuracy: 0.1003 - val_loss: 2.3041 - val_accuracy: 0.1050


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 179us/sample - loss: 0.7790 - accuracy: 0.7081 - val_loss: 0.5839 - val_accuracy: 0.7687
Epoch 2/3
54000/54000 [==============================] - 9s 172us/sample - loss: 0.5467 - accuracy: 0.7925 - val_loss: 0.4995 - val_accuracy: 0.8098
Epoch 3/3
54000/54000 [==============================] - 9s 174us/sample - loss: 0.4697 - accuracy: 0.8293 - val_loss: 0.4605 - val_accuracy: 0.8288


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 186us/sample - loss: 2.3040 - accuracy: 0.0997 - val_loss: 2.3030 - val_accuracy: 0.1050
Epoch 2/3
54000/54000 [==============================] - 10s 177us/sample - loss: 2.3039 - accuracy: 0.0984 - val_loss: 2.3033 - val_accuracy: 0.0973
Epoch 3/3
54000/54000 [==============================] - 10s 178us/sample - loss: 2.3041 - accuracy: 0.0981 - val_loss: 2.3036 - val_accuracy: 0.0985


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 180us/sample - loss: 2.3042 - accuracy: 0.0993 - val_loss: 2.3042 - val_accuracy: 0.1008
Epoch 2/3
54000/54000 [==============================] - 9s 169us/sample - loss: 2.3040 - accuracy: 0.1002 - val_loss: 2.3045 - val_accuracy: 0.1003
Epoch 3/3
54000/54000 [==============================] - 9s 171us/sample - loss: 2.3041 - accuracy: 0.0966 - val_loss: 2.3033 - val_accuracy: 0.0985


INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        76832     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1239120   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,317,722
Trainable params: 1,317,722
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 10s 178us/sample - loss: 0.4236 - accuracy: 0.8488 - val_loss: 0.4004 - val_accuracy: 0.8553
Epoch 5/10
54000/54000 [==============================] - 9s 172us/sample - loss: 0.3991 - accuracy: 0.8588 - val_loss: 0.3874 - val_accuracy: 0.8607
Epoch 6/10
54000/54000 [==============================] - 9s 164us/sample - loss: 0.3794 - accuracy: 0.8646 - val_loss: 0.3853 - val_accuracy: 0.8602
Epoch 7/10
54000/54000 [==============================] - 9s 169us/sample - loss: 0.3640 - accuracy: 0.8688 - val_loss: 0.3758 - val_accuracy: 0.8575
Epoch 8/10
54000/54000 [==============================] - 9s 168us/sample - loss: 0.3481 - accuracy: 0.8730 - val_loss: 0.3561 - val_accuracy: 0.8712
Epoch 9/10
54000/54000 [==============================] - 9s 170us/sample - loss: 0.3345 - accuracy: 0.8778 - val_loss: 0.3442 - val_accuracy: 0.8735
Epoch 10/10
54000/54000 [=========================